# RSLC to GSLC
- This notebook converts NISAR RSLC data to GSLC and then RSLC data by running ISCE3's `gslc.py`.
- Uses the `isce3_src` kernel (created using [Create_Environments.ipynb](https://github.com/isce-framework/sds-ondemand/blob/main/environments/Create_Environments.ipynb)).
- Can be ran locally (preferably on a GPU instance), or as a PCM job.

# Parameters
This cell is marked `parameters`, indicating the variables within can substituted when running this notebook via `papermill`.
- `data_link`: S3 url to the NISAR RSLC data to be converted into GSLC.
- `dem_s3_url`: S3 url to the DEM file to download.
- `gpu_enabled`: `1` to run using the GPU, `0` to use CPU instead. **Keep in mind that while disabling the GPU processing allows this notebook to be ran on an instance without a GPU, that does not guarantee the instance this notebook is running on is a non-GPU instance.** To run on a non-GPU instance on PCM, submit the job to a CPU-only queue.
- `gslc_config`: The runconfig passed to `focus.py`.

### Upload parameters (PCM only)
- `timestamp`: A time-string of the format `%Y%m%dT%H%M%S` indicating the time at which the job was submitted. This helps `pcm.py` find where the results of this job will be submitted when this notebook is ran as a PCM job.

In [1]:
data_link = 's3://nisar-st-data-ondemand/ALOS-1-data/RSLC/ALPSRP274410710-L1.0.h5' # string
dem_s3_url = 's3://nisar-st-data-ondemand/DEM-static/dem.tiff' # string
gpu_enabled = 1 # boolean
gslc_config = '' # string
timestamp = '20240131T0123456' # string

# hysds specifications
_time_limit = 86400
_soft_time_limit = 86400
_disk_usage = '10GB'
_submission_type = 'iteration'
_label = 'RSLC to GSLC PGE'

### Pre-processing of the Parameters to convert numbers or words into `boolean` True and False values.

In [2]:
# Convert boolean parameters because they must be specified as strings
try:
    if not isinstance(gpu_enabled, bool):
        gpu_enabled = int(gpu_enabled) > 0
except ValueError:
    if instance(gpu_enabled, str):
        gpu_enabled = gpu_enabled.lower() == 'true'
    else:
        gpu_enabled = False
print(f'{gpu_enabled=}')

# Extraneous parameters
focus_config = '' # string
insar_config = '' # string

gpu_enabled=True


# Functions for loading runconfig files and downloading from S3 buckets

In [3]:
import os
import yaml
import asf_search as asf
import boto3
import aws_uploader

WORKING_DIR = os.getcwd()
HOME_DIR = os.environ['HOME']
NOTEBOOK_PGE_DIR = os.environ.get('NOTEBOOK_PGE_DIR', WORKING_DIR)
ISCE3_BUILD_DIR = os.environ.get('ISCE3_BUILD_DIR', f'{HOME_DIR}/isce3/build')

DOWNLOAD_DIR = os.path.join(WORKING_DIR, 'downloads')
EXTRACT_DIR = os.path.join(WORKING_DIR, 'alos_data')
OUTPUT_DIR = os.path.join(WORKING_DIR, 'output')
PRODUCT_DIR = os.path.join(WORKING_DIR, 'product_path')

os.makedirs(DOWNLOAD_DIR, exist_ok=True)
os.makedirs(EXTRACT_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)

# RSLC
if focus_config == '':
    with open(os.path.join(NOTEBOOK_PGE_DIR, '../templates/focus.yaml'), 'r') as f:
        FOCUS_YML = yaml.safe_load(f)
else:
    print('Using custom focus.py run config...')
    FOCUS_YML = yaml.safe_load(focus_config)
    
# GSLC
if gslc_config == '':
    with open(os.path.join(NOTEBOOK_PGE_DIR, '../templates/gslc.yaml'), 'r') as f:
        GSLC_YML = yaml.safe_load(f)
else:
    print('Using custom gslc.py run config...')
    GSLC_YML = yaml.safe_load(gslc_config)

# INSAR
if insar_config == '':
    with open(os.path.join(NOTEBOOK_PGE_DIR, '../templates/insar.yaml'), 'r') as f:
        INSAR_YML = yaml.safe_load(f)
else:
    print('Using custom insar.py run config...')
    INSAR_YML = yaml.safe_load(insar_config)

print(WORKING_DIR)

/home/jovyan/alos-to-insar/notebook_pges


# Run ISCE3 Python Scripts
This cell runs the python scripts:
- `gslc.py`: Converts NISAR RSLC -> GSLC

In [4]:
from urllib.parse import urlparse
import isce3_config

# Download the specified L0B data
target_path = os.path.join(DOWNLOAD_DIR, os.path.basename(urlparse(data_link).path))
if not os.path.exists(target_path):
    print(f'Downloading {target_path} from S3 bucket at {data_link}...')
    aws_uploader.AWS.download_s3(data_link, target_path)
else:
    print(f'{target_path} already exists, skipping download...')

# Download the DEM locally
dem_f = isce3_config.download_dem(dem_s3_url, DOWNLOAD_DIR)

# Run gslc.py
yml_path = os.path.join(OUTPUT_DIR, 'gslc_final.yaml')
output_name = os.path.basename(os.path.splitext(target_path)[0])
output_f = os.path.join(PRODUCT_DIR, f'{output_name}_GSLC.h5')
isce3_config.write_gslc_config(GSLC_YML, target_path, dem_f, yml_path, gpu_enabled, output_f)
print(f'Executing:\n    mamba run -n isce3_src python {ISCE3_BUILD_DIR}/packages/nisar/workflows/gslc.py {yml_path} --no-log')
!mamba run -n isce3_src python {ISCE3_BUILD_DIR}/packages/nisar/workflows/gslc.py {yml_path} --no-log

/home/jovyan/alos-to-insar/notebook_pges/downloads/ALPSRP274410710-L1.0.h5 already exists, skipping download...
/home/jovyan/alos-to-insar/notebook_pges/downloads/dem.tiff already exists, skipping download...
Downloaded DEM: /home/jovyan/alos-to-insar/notebook_pges/downloads/dem.tiff
Executing:
    mamba run -n isce3_src python /home/jovyan/isce3/build/packages/nisar/workflows/gslc.py /home/jovyan/alos-to-insar/notebook_pges/output/gslc_final.yaml --no-log
journal: preparing HDF5
journal: successfully ran h5_prep
journal: starting geocode SLC
journal: running geocode SLC array block 1 of 130
journal: no radar data found for block 1 of 130
journal: running geocode SLC array block 14 of 130
journal: no radar data found for block 14 of 130
journal: running geocode SLC array block 27 of 130
journal: no radar data found for block 27 of 130
journal: running geocode SLC array block 40 of 130
journal: no radar data found for block 40 of 130
journal: running geocode SLC array block 53 of 130
jo

# Automatic Stage Out for PCM Jobs
This routine creates a folder which matches the regex:

#### GSLC
**Example:** `NISAR_L2_PR_GSLC_000_000_Z_000_0000_HH_20240131T000000_20250131T000000_0A1B2C_A_F_0000`
```
(?P<id>NISAR_L2_(?P<processing_type>[P|U]R)_GSLC_(?P<cycle_number>\d{3})_(?P<relative_orbit_number>\d{3})_(?P<Direction>\w)_(?P<TrackFrame>\d{3})_(?P<radar_processing_mode>\d{4})_(?P<Polarization>\w{2})_(?P<RadarStartDateTime>(?P<year>\d{4})(?P<month>\d{2})(?P<day>\d{2})T\d{6})_(?P<RadarStopDateTime>\d{8}T\d{6})_(?P<composite_release_id>\w{6})_(?P<Fidelity>\w)_(?P<CoverageIndicator>[F|P])_(?P<product_counter>\d{3}))$
```
This enables PCM to automatically upload the results of this notebook to the `s3://nisar-{system_name}-rs-ondemand` bucket.

`system_name` is one of the following:
- `st` (for Science Team)
- `adt` (for ADT)
- `iot` (for IOT)

In [5]:
import h5py
import json

# Grab some appropriate values for PCM to automatically stage out
polarization = 'HH'
regex_name = isce3_config.GSLC_FORMAT.format(
    polarization=polarization,
    timestamp=timestamp)
auto_output_dir = os.path.join(WORKING_DIR, regex_name)

# Create the automatic stage-out directory
!mkdir -p {auto_output_dir}
!cp {WORKING_DIR}/*-output.ipynb {auto_output_dir}
!cp {WORKING_DIR}/*.txt {auto_output_dir} 
!cp {WORKING_DIR}/*.json {auto_output_dir}
!mv {yml_path} {auto_output_dir}
!mv {output_f} {auto_output_dir}

# Move the new path variables to avoid confusion
yml_path = os.path.join(auto_output_dir, os.path.basename(yml_path))
output_f = os.path.join(auto_output_dir, os.path.basename(output_f))

# Create metadata files for the automatic stage-out directory
with open(os.path.join(auto_output_dir, f'{regex_name}.met.json'), 'w', encoding='utf-8') as f:
    content = {
        'polarization': polarization,
    }
    f.write(json.dumps(content))
with open(os.path.join(auto_output_dir, f'{regex_name}.dataset.json'), 'w', encoding='utf-8') as f:
    content = {
         'version': 'v1.0',
         'label': 'This is purely an EXAMPLE metadata file, the values in this file are not representative of this product.',
         'location': {
           'type': 'polygon',
           'coordinates': [
             [
                [-122.9059682940358,40.47090915967475],
                [-121.6679748715316,37.84406528996276],
                [-120.7310161872557,38.28728069813177],
                [-121.7043611684245,39.94137004454238],
                [-121.9536916840953,40.67097860759095],
                [-122.3100379696548,40.7267890636145],
                [-122.7640648263371,40.5457010812299],
                [-122.9059682940358,40.47090915967475]
              ]
            ]
        },
        'starttime': '2017-01-01T00:00:00',
        'endtime': '2017-01-01T00:05:00',
    }
    f.write(json.dumps(content))

cp: cannot stat ‘/home/jovyan/alos-to-insar/notebook_pges/*-output.ipynb’: No such file or directory
cp: cannot stat ‘/home/jovyan/alos-to-insar/notebook_pges/*.txt’: No such file or directory
cp: cannot stat ‘/home/jovyan/alos-to-insar/notebook_pges/*.json’: No such file or directory
